In [ ]:
import requests
from bs4 import BeautifulSoup
import lxml
import time
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Session
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import Mapped
import json

In [ ]:
with open("config.json", "r") as configFile:
    config = json.loads(configFile.read())


In [ ]:
class Base(DeclarativeBase):
    pass

class vacancies(Base):
    __tablename__ = 'vacancies'
 
    key: Mapped[int] = mapped_column(primary_key=True)
    company_name: Mapped[str]
    position: Mapped[str]
    job_description: Mapped[str] 
    key_skills: Mapped[str]

engine = create_engine(config['dbConnect'])
Base.metadata.create_all(engine)

In [ ]:
def parsingVacancyList (url,page):

    url_params = {
    "text": "middle python",
    "search_field": "name",
    "per_page": "20",
    "page": page
    }

    result = requests.get(config['urlSearch'], headers=config['user_agent'], params=url_params)
    if result.status_code == 200:
        soup = BeautifulSoup(result.content.decode(), "html.parser")
        vacancyList = soup.find_all('a', attrs={'data-qa': 'serp-item__title'})
        return vacancyList
    else: 
        print(f"Старница списка вакансий не загружена. Код ошибки: {result.status_code}. URL: {url}")
        return None

In [ ]:
def parsingVacancy(url):
    tagVacancyList = []
    result = requests.get(url, headers=config['user_agent'])
    if result.status_code == 200:
        soup = BeautifulSoup(result.content.decode(), "html.parser")
        nameVacancy = soup.find('h1').text
        nameCompany = soup.find('a', attrs={'data-qa': 'vacancy-company-name'}).text
        descriptionVacancy = soup.find('div', attrs={'data-qa': 'vacancy-description'}).text
        for tag in soup.find_all('div', attrs={'data-qa': 'bloko-tag bloko-tag_inline skills-element'}):
            tagVacancyList.append(tag.text)
        tagVacancy = ', '.join(tagVacancyList)

        if tagVacancy:

            insert = vacancies(company_name=nameCompany,
                         position=nameVacancy, 
                         job_description=descriptionVacancy, 
                         key_skills=tagVacancy)
            return insert
        else:
            return None
    else: 
        print(f"Старница вакансии не загружена. Код ошибки: {result.status_code}. URL: {url}")
        return None

In [102]:

list = []
vacancyCount = 0
page = 0

while (vacancyCount <= 100 or page <=20):
    vacancyList = parsingVacancyList (config['urlSearch'],page)
    if vacancyList is not None:
        if len(vacancyList) == 0: 
            break
        for vacancy in vacancyList:
            insert = parsingVacancy(vacancy.attrs['href'])
            if insert is not None:
                list.append(insert)
                vacancyCount += 1
            time.sleep(5)
    page += 1
    print(f"Старница: {page}. Вакансий: {vacancyCount}")
    

AttributeError: 'NoneType' object has no attribute 'text'

In [101]:
len(list)

7

In [ ]:
with Session(engine) as session:
    session.add_all(list)
    session.commit()